In [23]:
import pandas as pd
import numpy as np
import tifffile as tiff
import os
from skimage.measure import label, regionprops

# Constants for conversions
PIXELS_PER_MICRON = 9.3283
Z_SLICE_HEIGHT_MICRON = 1.8  # Micrometers per z-slice
PIXEL_AREA_TO_MICRON_SQ = (1 / PIXELS_PER_MICRON) ** 2  # Conversion factor from pixel² to µm²

# Define file paths
mask_folder = r"\\bifchem.z.science.ru.nl\bifchem\Projects Hansen\Lab Members folders\CG\Experiments\CHX_CG1\CHX_CG1_100\processed_images\+OPP+RB\488_masks"
binary_folder = r"\\bifchem.z.science.ru.nl\bifchem\Projects Hansen\Lab Members folders\CG\Experiments\CHX_CG1\CHX_CG1_100\processed_images\+OPP+RB\488_binary_stacks"
C1_folder = r"\\bifchem.z.science.ru.nl\bifchem\Projects Hansen\Lab Members folders\CG\Experiments\CHX_CG1\CHX_CG1_100\processed_images\+OPP+RB\8-bit_Z_proj_405_525"
C2_folder = r"\\bifchem.z.science.ru.nl\bifchem\Projects Hansen\Lab Members folders\CG\Experiments\CHX_CG1\CHX_CG1_100\processed_images\+OPP+RB\8-bit_Z_proj_488"
C3_folder = r"\\bifchem.z.science.ru.nl\bifchem\Projects Hansen\Lab Members folders\CG\Experiments\CHX_CG1\CHX_CG1_100\processed_images\+OPP+RB\8-bit_Z_proj_647"

mask_file = os.path.join(mask_folder, "C2-PANC1_rep2_pos_10_8-bit_Z_proj_mask.tif")
binary_file = os.path.join(binary_folder, "MASK_C2-PANC1_rep2_pos_10_8-bit.tif")
C1_file = os.path.join(C1_folder, "C1-PANC1_rep2_pos_10_8-bit_Z_proj.tif")
C2_file = os.path.join(C2_folder, "C2-PANC1_rep2_pos_10_8-bit_Z_proj.tif")
C3_file = os.path.join(C3_folder, "C3-PANC1_rep2_pos_10_8-bit_Z_proj.tif")

# Load the mask and binary images
mask = tiff.imread(mask_file)
binary = tiff.imread(binary_file)
C1_405 = tiff.imread(C1_file)
C2_488 = tiff.imread(C2_file)
C3_647 = tiff.imread(C3_file)

# Step 1: Label the mask
label_mask = label(mask)

# Step 2: Get properties of labeled regions
props = regionprops(label_mask)

# Filter labels by area > 5000
filtered_labels = [prop for prop in props if prop.area > 5000]

# Step 3: Calculate the volume for each label and store average intensity
label_data = []

for prop in filtered_labels:
    label_id = prop.label
    coords = prop.coords  # Get the coordinates of the current label in the mask

    # Calculate total volume for the label
    total_volume = 0.0
    for z_slice in range(binary.shape[0]):  # Iterate over each z-slice
        binary_slice = binary[z_slice]
        label_mask_in_z = binary_slice[coords[:, 0], coords[:, 1]]
        non_zero_area_pixels = np.count_nonzero(label_mask_in_z)
        non_zero_area_microns_sq = non_zero_area_pixels * PIXEL_AREA_TO_MICRON_SQ
        slice_volume = non_zero_area_microns_sq * Z_SLICE_HEIGHT_MICRON
        total_volume += slice_volume

    # Extract intensities for each channel
    C1_intensities = C1_405[coords[:, 0], coords[:, 1]]
    C2_intensities = C2_488[coords[:, 0], coords[:, 1]]
    C3_intensities = C3_647[coords[:, 0], coords[:, 1]]

    # Calculate average intensity for each channel within the label
    avg_C1_intensity = np.mean(C1_intensities)
    avg_C2_intensity = np.mean(C2_intensities)
    avg_C3_intensity = np.mean(C3_intensities)

    # Calculate normalized intensities by dividing average intensities by volume
    norm_C1_intensity = avg_C1_intensity / total_volume if total_volume != 0 else 0
    norm_C2_intensity = avg_C2_intensity / total_volume if total_volume != 0 else 0
    norm_C3_intensity = avg_C3_intensity / total_volume if total_volume != 0 else 0

    # Store the results as a dictionary
    label_data.append({
        'Label_ID': label_id,
        'Volume_µm³': total_volume,
        'C1_405_avg_intensity': avg_C1_intensity,
        'C2_488_avg_intensity': avg_C2_intensity,
        'C3_647_avg_intensity': avg_C3_intensity,
        'C1_405_norm_intensity': norm_C1_intensity,
        'C2_488_norm_intensity': norm_C2_intensity,
        'C3_647_norm_intensity': norm_C3_intensity
    })

# Step 4: Convert the results into a DataFrame
df = pd.DataFrame(label_data)

# Print the DataFrame
print(df)

# Optionally, save to a CSV file
df.to_csv("label_volumes_and_intensities_with_normalization_SP.csv", index=False)


     Label_ID   Volume_µm³  C1_405_avg_intensity  C2_488_avg_intensity  \
0           1   634.695349             18.533068             70.156731   
1           2   622.015095             17.300635             69.988181   
2           4  5768.853862             26.641148            107.820691   
3           5  5506.850430             21.421931             93.029516   
4           6  3329.383888             24.796344            111.385939   
..        ...          ...                   ...                   ...   
118       141  3333.396888             26.667709            106.740089   
119       142  1672.924801             20.817151             79.907728   
120       143  1034.733590             20.240302             79.537057   
121       144  2413.013130             23.965206             93.349665   
122       145  1125.750099             22.203145             91.206604   

     C3_647_avg_intensity  C1_405_norm_intensity  C2_488_norm_intensity  \
0               61.880957           

In [1]:
import pandas as pd
import numpy as np
import tifffile as tiff
import os
from skimage.measure import label, regionprops

# Constants for conversions
PIXELS_PER_MICRON = 9.3283
Z_SLICE_HEIGHT_MICRON = 1.8  # Micrometers per z-slice
PIXEL_AREA_TO_MICRON_SQ = (1 / PIXELS_PER_MICRON) ** 2  # Conversion factor from pixel² to µm²

# Define file paths
mask_folder = r"\\bifchem.z.science.ru.nl\bifchem\Projects Hansen\Lab Members folders\CG\Experiments\CHX_CG1\CHX_CG1_100\processed_images\+OPP+RB\488_masks"
binary_folder = r"\\bifchem.z.science.ru.nl\bifchem\Projects Hansen\Lab Members folders\CG\Experiments\CHX_CG1\CHX_CG1_100\processed_images\+OPP+RB\488_binary_stacks"
C1_folder = r"\\bifchem.z.science.ru.nl\bifchem\Projects Hansen\Lab Members folders\CG\Experiments\CHX_CG1\CHX_CG1_100\processed_images\+OPP+RB\8-bit_Z_proj_405_525"
C2_folder = r"\\bifchem.z.science.ru.nl\bifchem\Projects Hansen\Lab Members folders\CG\Experiments\CHX_CG1\CHX_CG1_100\processed_images\+OPP+RB\8-bit_Z_proj_488"
C3_folder = r"\\bifchem.z.science.ru.nl\bifchem\Projects Hansen\Lab Members folders\CG\Experiments\CHX_CG1\CHX_CG1_100\processed_images\+OPP+RB\8-bit_Z_proj_647"

# List to store data for the DataFrame
label_data = []

# Step 1: Iterate through each mask file in the mask folder
for mask_filename in os.listdir(mask_folder):
    if mask_filename.endswith('_mask.tif'):
        # Construct full file path for the mask
        mask_file = os.path.join(mask_folder, mask_filename)
        
        # Replace hyphens with underscores in the filename
        mask_filename = mask_filename.replace('-', '_')
        
        # Derive the experiment name from the modified mask file name
        experiment_name_parts = mask_filename.split('_')[1:5] 
        experiment_name = '_'.join(experiment_name_parts) # Now using the modified filename
        print(experiment_name)  # Print to check the output

        
        # Construct the corresponding binary file name
        binary_file = os.path.join(binary_folder, f'MASK_C2-{experiment_name}_8-bit.tif')  # Removing '_mask.tif'
        
        # Load the mask and binary images
        mask = tiff.imread(mask_file)
        binary = tiff.imread(binary_file)

        # Step 2: Label the mask
        label_mask = label(mask)

        # Step 3: Get properties of labeled regions
        props = regionprops(label_mask)

        # Filter labels by area > 5000
        filtered_labels = [prop for prop in props if prop.area > 5000]

        # Step 4: Calculate the volume for each label and store average intensity
        for prop in filtered_labels:
            label_id = prop.label
            coords = prop.coords  # Get the coordinates of the current label in the mask

            # Calculate total volume for the label
            total_volume = 0.0
            for z_slice in range(binary.shape[0]):  # Iterate over each z-slice
                binary_slice = binary[z_slice]
                label_mask_in_z = binary_slice[coords[:, 0], coords[:, 1]]
                non_zero_area_pixels = np.count_nonzero(label_mask_in_z)
                non_zero_area_microns_sq = non_zero_area_pixels * PIXEL_AREA_TO_MICRON_SQ
                slice_volume = non_zero_area_microns_sq * Z_SLICE_HEIGHT_MICRON
                total_volume += slice_volume

            # Extract intensities for each channel
            C1_file = os.path.join(C1_folder, f"C1-{experiment_name}_8-bit_Z_proj.tif")
            C2_file = os.path.join(C2_folder, f"C2-{experiment_name}_8-bit_Z_proj.tif")
            C3_file = os.path.join(C3_folder, f"C3-{experiment_name}_8-bit_Z_proj.tif")

            # Load channel images
            C1_405 = tiff.imread(C1_file)
            C2_488 = tiff.imread(C2_file)
            C3_647 = tiff.imread(C3_file)

            # Calculate average intensity for each channel within the label
            C1_intensities = C1_405[coords[:, 0], coords[:, 1]]
            C2_intensities = C2_488[coords[:, 0], coords[:, 1]]
            C3_intensities = C3_647[coords[:, 0], coords[:, 1]]

            avg_C1_intensity = np.mean(C1_intensities)
            avg_C2_intensity = np.mean(C2_intensities)
            avg_C3_intensity = np.mean(C3_intensities)

            # Calculate normalized intensities by dividing average intensities by volume
            norm_C1_intensity = avg_C1_intensity / total_volume if total_volume != 0 else 0
            norm_C2_intensity = avg_C2_intensity / total_volume if total_volume != 0 else 0
            norm_C3_intensity = avg_C3_intensity / total_volume if total_volume != 0 else 0

            # Store the results as a dictionary including the experiment name
            label_data.append({
                'Label_ID': label_id,
                'CellLine_replicate_position': experiment_name,  # Store the experiment name
                'Volume_µm³': total_volume,
                'C1_405_avg_intensity': avg_C1_intensity,
                'C2_488_avg_intensity': avg_C2_intensity,
                'C3_647_avg_intensity': avg_C3_intensity,
                'C1_405_norm_intensity': norm_C1_intensity,
                'C2_488_norm_intensity': norm_C2_intensity,
                'C3_647_norm_intensity': norm_C3_intensity
            })

# Step 5: Convert the results into a DataFrame
df = pd.DataFrame(label_data)

# Print the DataFrame
print(df)

# Optionally, save to a CSV file
output_folder = r"\\bifchem.z.science.ru.nl\bifchem\Projects Hansen\Lab Members folders\CG\Experiments\CHX_CG1\CHX_CG1_100\processed_images\+OPP+RB\results"
df.to_csv(output_folder + "\label_volumes_and_intensities_with_normalization.csv", index=False)


HCT116_rep1_pos_10
HCT116_rep1_pos_11
HCT116_rep1_pos_12
HCT116_rep1_pos_13
HCT116_rep1_pos_3
HCT116_rep1_pos_4
HCT116_rep1_pos_5
HCT116_rep1_pos_6
HCT116_rep1_pos_7
HCT116_rep1_pos_8
HCT116_rep1_pos_9
HCT116_rep2_pos_10
HCT116_rep2_pos_1
HCT116_rep2_pos_2
HCT116_rep2_pos_3
HCT116_rep2_pos_4
HCT116_rep2_pos_5
HCT116_rep2_pos_6
HCT116_rep2_pos_7
HCT116_rep2_pos_8
HCT116_rep2_pos_9
HEK_rep1_pos_10
HEK_rep1_pos_11
HEK_rep1_pos_1
HEK_rep1_pos_2
HEK_rep1_pos_3
HEK_rep1_pos_4
HEK_rep1_pos_5
HEK_rep1_pos_7
HEK_rep1_pos_8
HEK_rep1_pos_9
HEK_rep2_pos_10
HEK_rep2_pos_1
HEK_rep2_pos_2
HEK_rep2_pos_3
HEK_rep2_pos_4
HEK_rep2_pos_5
HEK_rep2_pos_6
HEK_rep2_pos_7
HEK_rep2_pos_8
HEK_rep2_pos_9
HeLa_rep1_pos_10
HeLa_rep1_pos_1
HeLa_rep1_pos_2
HeLa_rep1_pos_3
HeLa_rep1_pos_4
HeLa_rep1_pos_5
HeLa_rep1_pos_6
HeLa_rep1_pos_7
HeLa_rep1_pos_8
HeLa_rep1_pos_9
HeLa_rep2_pos_10
HeLa_rep2_pos_1
HeLa_rep2_pos_2
HeLa_rep2_pos_3
HeLa_rep2_pos_4
HeLa_rep2_pos_5
HeLa_rep2_pos_6
HeLa_rep2_pos_7
HeLa_rep2_pos_8
HeLa_rep